# Sports Betting Data Cleaning and Basic Data Driven Strategy #1

### Package Imports

In [1]:
from OddsJamClient import OddsJamClient;
from dotenv import load_dotenv
import os, requests, datetime, json
import pandas as pd

### Load Env Vars

In [2]:
load_dotenv()
ODDSJAM_API_KEY = os.getenv("ODDSJAM_API_KEY")

### Initialize Clients

In [3]:
Client = OddsJamClient(ODDSJAM_API_KEY);
Client.UseV2();

### Get Games For Today

In [4]:
def get_games_from_league_as_dataframe(league="nba", sport="basketball", game_date=None):
    # Define the endpoint URL
    end_point = 'https://api-external.oddsjam.com/api/v2/games'
    
    # Set up the headers with the API key and content type
    headers = {
        # 'x-api-key': ODDSJAM_API_KEY,
        'Content-Type': 'application/json'
    }

    if not game_date : 
        game_date = datetime.datetime.now().strftime('%Y-%m-%d')
    else:
        # Parse the string into a datetime object
        parsed_date = datetime.datetime.strptime(game_date, '%m/%d/%Y')
        # Format the datetime object to the desired string format
        game_date = parsed_date.strftime('%Y-%m-%d')

    
    # Set up the data payload with the parameters
    data = {
        'league': league,
        'sport': sport,
        "include_team_info": True,
        "key": ODDSJAM_API_KEY,
        "game_date": game_date
    }
    
    # Make the POST request to the API
    response = requests.get(end_point, headers=headers, params=data)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Here you would typically convert the response to a DataFrame
        df = pd.DataFrame(response.json()["data"])
        # update the time
        df['start_date'] = pd.to_datetime(df['start_date']).dt.strftime('%Y-%m-%d %H:%M:%S')
        df = df.sort_values(by='start_date').reset_index(drop=True)
        # Now, we extract the team ids from the home_team_info and away_team_info dictionaries
        # and then drop these columns from the DataFrame
        df['home_team_id'] = df['home_team_info'].apply(lambda x: x['id'])
        df['away_team_id'] = df['away_team_info'].apply(lambda x: x['id'])

        # Drop the now-redundant '_team_info' columns
        df = df.drop(columns=['home_team_info', 'away_team_info'])
        return df
    else:
        print(f"Error: {response.status_code}")
        # You could also handle errors here


In [5]:
games_df = get_games_from_league_as_dataframe(league="NBA", sport="basketball")



In [6]:
display(games_df)

,id,start_date,home_team,away_team,is_live,is_popular,tournament,status,sport,league,home_team_id,away_team_id
0,19957-14251-2024-01-26,2024-01-26 19:00:00,Atlanta Hawks,Dallas Mavericks,False,False,None,unplayed,basketball,NBA,B59C1C735494,7165DAB9CAE4
1,32825-24860-2024-01-26,2024-01-26 19:00:00,Indiana Pacers,Phoenix Suns,False,False,None,unplayed,basketball,NBA,F342C875E571,9BF9A5FD18B1
2,35142-17844-2024-01-26,2024-01-26 19:00:00,Charlotte Hornets,Houston Rockets,False,False,None,unplayed,basketball,NBA,C65360931346,DF2D9E9E8E20
3,27843-35775-2024-01-26,2024-01-26 19:30:00,Toronto Raptors,Los Angeles Clippers,False,False,None,unplayed,basketball,NBA,417F4FFF4625,3E1EA77686B9
4,11464-12967-2024-01-26,2024-01-26 20:00:00,Milwaukee Bucks,Cleveland Cavaliers,False,False,None,unplayed,basketball,NBA,14682EF45C4D,D5348BDFEBCC
5,22796-30354-2024-01-26,2024-01-26 20:00:00,Memphis Grizzlies,Orlando Magic,False,False,None,unplayed,basketball,NBA,2C653B0A5BBF,CC72CD00EB95
6,86230-19432-2024-01-26,2024-01-26 20:00:00,New Orleans Pelicans,Oklahoma City Thunder,False,False,None,unplayed,basketball,NBA,6C595835C85B,D8EC6878976A
7,38874-32634-2024-01-26,2024-01-26 21:30:00,San Antonio Spurs,Portland Trail Blazers,False,False,None,unplayed,basketball,NBA,E89F51275352,0054C2679F77


### Segment The Games Into Sessions 
- Sessions are defined by games grouped together that start within 2 hours of each other
- no two sessions will share the same game; they're all exclusive sessions.

In [7]:
def segment_games_into_sessions(games_df):
    # Ensure start_date is a datetime
    games_df['start_date'] = pd.to_datetime(games_df['start_date'])
    
    # Sort the games by start time
    sorted_games_df = games_df.sort_values(by='start_date').reset_index(drop=True)
    
    # List to hold all game sessions
    sessions = []
    # Temporary list to store games for the current session
    current_session = []
    # Start time of the current session
    session_start = None
    
    for _, game in sorted_games_df.iterrows():
        game_start = game['start_date']
        
        # If current session is empty or game starts within 2 hours of the session start
        if not current_session or (game_start - session_start).total_seconds() <= 2 * 3600:
            # Add game to current session
            current_session.append(game)
            # If this is the first game in the session, set the session start time
            if not session_start:
                session_start = game_start
        else:
            # If game starts more than 2 hours after the session start, save and reset the session
            sessions.append(current_session)
            current_session = [game]
            session_start = game_start
    
    # Add the last session if it exists
    if current_session:
        sessions.append(current_session)
    
    # Convert sessions to DataFrame list for better handling
    session_dfs = [pd.DataFrame(session) for session in sessions]
    
    return session_dfs

In [30]:
# Example usage:
# Assuming games_df is your DataFrame containing the games information.
segmented_sessions = segment_games_into_sessions(games_df)

# You can then access each session DataFrame with segmented_sessions[0], segmented_sessions[1], etc.
len(segmented_sessions)

2

### Get Players From A Specific Team

In [31]:
def get_players_from_specific_team(team_id: str, league="nba", sport="basketball") -> pd.DataFrame:
    # Define the endpoint URL
    end_point = 'https://api-external.oddsjam.com/api/v2/players/list'
    
    # Set up the headers with the API key
    headers = {
        'Content-Type': 'application/json'
    }
    
    # Set up the parameters with the team_id
    params = {
        'team': team_id,
        'page': 1,
        # "league": league,
        # "sport": sport,
        'key': ODDSJAM_API_KEY
    }
    
    # Make the GET request to the API
    response = requests.get(end_point, headers=headers, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the JSON response to a DataFrame
        data = response.json()["data"]
        df = pd.DataFrame(data)
        print(f'total pages: {response.json()["total_pages"]}')
        return df
    else:
        print(f"Error: {response.status_code}")
        return pd.DataFrame()

### Testing That The Functionality Works As Envisioned
* Need to get the game_id from the segmented_sessions  

In [32]:
display(segmented_sessions[0])

,id,start_date,home_team,away_team,is_live,is_popular,tournament,status,sport,league,home_team_id,away_team_id
0,19957-14251-2024-01-26,2024-01-26 19:00:00,Atlanta Hawks,Dallas Mavericks,False,False,None,unplayed,basketball,NBA,B59C1C735494,7165DAB9CAE4
1,32825-24860-2024-01-26,2024-01-26 19:00:00,Indiana Pacers,Phoenix Suns,False,False,None,unplayed,basketball,NBA,F342C875E571,9BF9A5FD18B1
2,35142-17844-2024-01-26,2024-01-26 19:00:00,Charlotte Hornets,Houston Rockets,False,False,None,unplayed,basketball,NBA,C65360931346,DF2D9E9E8E20
3,27843-35775-2024-01-26,2024-01-26 19:30:00,Toronto Raptors,Los Angeles Clippers,False,False,None,unplayed,basketball,NBA,417F4FFF4625,3E1EA77686B9
4,11464-12967-2024-01-26,2024-01-26 20:00:00,Milwaukee Bucks,Cleveland Cavaliers,False,False,None,unplayed,basketball,NBA,14682EF45C4D,D5348BDFEBCC
5,22796-30354-2024-01-26,2024-01-26 20:00:00,Memphis Grizzlies,Orlando Magic,False,False,None,unplayed,basketball,NBA,2C653B0A5BBF,CC72CD00EB95
6,86230-19432-2024-01-26,2024-01-26 20:00:00,New Orleans Pelicans,Oklahoma City Thunder,False,False,None,unplayed,basketball,NBA,6C595835C85B,D8EC6878976A


In [33]:
get_players_from_specific_team(segmented_sessions[1].iloc[0]['home_team_id'])

total pages: 1


,id,player_name,first_name,last_name,team_name,team_id,number,position,age,height,weight,is_active,sport,league
0,7959261438C7,Blake Wesley,Blake,Wesley,San Antonio Spurs,E89F51275352,14,SG,21,77,181,True,basketball,NBA
1,97294E569CB7,Cedi Osman,Cedi,Osman,San Antonio Spurs,E89F51275352,16,SG,29,79,215,True,basketball,NBA
2,CD99BB67884D,Charles Bassey,Charles,Bassey,San Antonio Spurs,E89F51275352,28,C,24,83,235,True,basketball,NBA
3,E10320F559B1,David Duke Jr.,David,Duke Jr.,San Antonio Spurs,E89F51275352,7,SG,25,76,205,True,basketball,NBA
4,804B793FEEDB,Devin Vassell,Devin,Vassell,San Antonio Spurs,E89F51275352,24,SG,24,79,194,True,basketball,NBA
5,AACF1CBA1094,Devonte' Graham,Devonte',Graham,San Antonio Spurs,E89F51275352,4,SG,29,73,185,True,basketball,NBA
6,17624A972445,Dominick Barlow,Dominick,Barlow,San Antonio Spurs,E89F51275352,26,SF,21,81,214,True,basketball,NBA
7,CA04B2FDE9C2,Doug McDermott,Doug,McDermott,San Antonio Spurs,E89F51275352,17,PF,32,79,225,True,basketball,NBA
8,80FFC6ED3456,Jeremy Sochan,Jeremy,Sochan,San Antonio Spurs,E89F51275352,10,PF,21,81,230,True,basketball,NBA
9,99318830CCC3,Julian Champagnie,Julian,Champagnie,San Antonio Spurs,E89F51275352,30,SF,23,80,215,True,basketball,NBA


In [34]:
temp = get_players_from_specific_team(segmented_sessions[1].iloc[0]['away_team_id'])

total pages: 1


In [40]:
temp

,id,player_name,first_name,last_name,team_name,team_id,number,position,age,height,weight,is_active,sport,league
0,A08DF5DBBEE9,Anfernee Simons,Anfernee,Simons,Portland Trail Blazers,0054C2679F77,1,SG,25,75,185,True,basketball,NBA
1,54818F8E80E9,Deandre Ayton,Deandre,Ayton,Portland Trail Blazers,0054C2679F77,2,C,26,83,250,True,basketball,NBA
2,98CCB5584896,Duop Reath,Duop,Reath,Portland Trail Blazers,0054C2679F77,26,C,28,83,244,True,basketball,NBA
3,0B46AA021772,Ibou Badji,Ibou,Badji,Portland Trail Blazers,0054C2679F77,41,C,22,84,240,True,basketball,NBA
4,677724637AF2,Jabari Walker,Jabari,Walker,Portland Trail Blazers,0054C2679F77,34,PF,22,81,215,True,basketball,NBA
5,628DC7694C25,Jerami Grant,Jerami,Grant,Portland Trail Blazers,0054C2679F77,9,PF,30,80,220,True,basketball,NBA
6,DFCA987B0935,Justin Minaya,Justin,Minaya,Portland Trail Blazers,0054C2679F77,24,SF,25,79,210,True,basketball,NBA
7,0E43E155B0D5,Kris Murray,Kris,Murray,Portland Trail Blazers,0054C2679F77,8,SF,24,80,220,True,basketball,NBA
8,D580A3FB8020,Malcolm Brogdon,Malcolm,Brogdon,Portland Trail Blazers,0054C2679F77,11,PG,32,77,224,True,basketball,NBA
9,805BF30F4B70,Matisse Thybulle,Matisse,Thybulle,Portland Trail Blazers,0054C2679F77,4,SF,27,77,200,True,basketball,NBA


### Get All Available Player Props For Each Team, Given There's A PrizePicks Line

In [44]:
def get_all_player_odds(player_ids: list, sportsbooks: list, league="nba", sport="basketball"):
    all_data = []
    # Loop through the player_ids in steps of 5
    for i in range(0, len(player_ids), 3):
        batch_ids = player_ids[i:i+3]
        params = {
            'player_id': batch_ids,
            'sportsbook': sportsbooks,
            'league': league,
            'sport': sport,
            'key': ODDSJAM_API_KEY
        }
        response = requests.get('https://api-external.oddsjam.com/api/v2/game-odds', headers=headers, params=params)
        
        if response.status_code == 200:
            # Convert the JSON response to a DataFrame
            data = response.json()['data']
            df = pd.DataFrame(data)
            all_data.append(df)
        else:
            print(f"Error: {response.reason}")
            print(response.text)
            # Depending on your error handling preference, you may comment out the next line
            # to continue processing in case of an error, or leave it to stop the function.
            return pd.DataFrame() 

    # Concatenate all dataframes into a single dataframe
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

### Test That The Function Pulls Odds Correctly

In [48]:
id_list = temp['id'].tolist()
display(id_list)
odds = get_player_odds(id_list[:5], ["PrizePicks", "Underdog Fantasy", "Pinnacle", "FanDuel", "Draftkings", "BetMGM"])

['A08DF5DBBEE9',
 '54818F8E80E9',
 '98CCB5584896',
 '0B46AA021772',
 '677724637AF2',
 '628DC7694C25',
 'DFCA987B0935',
 '0E43E155B0D5',
 'D580A3FB8020',
 '805BF30F4B70',
 '9429EDA9E051',
 'CE0B782BFD67',
 '61A9758D002F',
 'C3F1E061CAD6',
 'D53995324E8F',
 'D210661887F5',
 'DD05A1E94051']

In [49]:
odds.shape

(1, 11)

In [50]:
books = odds.iloc[0].odds
books

[{'id': '38874-32634-2024-01-26:betmgm:first_basket:anfernee_simons',
  'sports_book_name': 'BetMGM',
  'name': 'Anfernee Simons',
  'price': 750.0,
  'timestamp': 1706283745.4306633,
  'bet_points': None,
  'is_main': True,
  'is_live': False,
  'market_name': 'First Basket',
  'market': 'first_basket',
  'home_rotation_number': None,
  'away_rotation_number': None,
  'deep_link_url': 'https://sports.<STATE>.betmgm.com/en/sports?options=15236618-1054986410--1239083334&type=Single',
  'player_id': 'A08DF5DBBEE9',
  'selection': 'Anfernee Simons',
  'normalized_selection': 'anfernee_simons',
  'selection_line': None,
  'selection_points': None},
 {'id': '38874-32634-2024-01-26:betmgm:first_basket:deandre_ayton',
  'sports_book_name': 'BetMGM',
  'name': 'Deandre Ayton',
  'price': 600.0,
  'timestamp': 1706283745.4306633,
  'bet_points': None,
  'is_main': True,
  'is_live': False,
  'market_name': 'First Basket',
  'market': 'first_basket',
  'home_rotation_number': None,
  'away_rotat

In [51]:
books_df = pd.DataFrame(books)
display(books_df.head())
print(books_df.shape)

,id,sports_book_name,name,price,timestamp,bet_points,is_main,is_live,market_name,market,home_rotation_number,away_rotation_number,deep_link_url,player_id,selection,normalized_selection,selection_line,selection_points
0,38874-32634-2024-01-26:betmgm:first_basket:anf...,BetMGM,Anfernee Simons,750.0,1.706284e+09,NaN,True,False,First Basket,first_basket,NaN,NaN,https://sports.<STATE>.betmgm.com/en/sports?op...,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,None,NaN
1,38874-32634-2024-01-26:betmgm:first_basket:dea...,BetMGM,Deandre Ayton,600.0,1.706284e+09,NaN,True,False,First Basket,first_basket,NaN,NaN,https://sports.<STATE>.betmgm.com/en/sports?op...,54818F8E80E9,Deandre Ayton,deandre_ayton,None,NaN
2,38874-32634-2024-01-26:betmgm:first_basket:jab...,BetMGM,Jabari Walker,900.0,1.706284e+09,NaN,True,False,First Basket,first_basket,NaN,NaN,https://sports.<STATE>.betmgm.com/en/sports?op...,677724637AF2,Jabari Walker,jabari_walker,None,NaN
3,38874-32634-2024-01-26:betmgm:player_assists:a...,BetMGM,Anfernee Simons Over 5.5,-120.0,1.706293e+09,5.5,True,False,Player Assists,player_assists,NaN,NaN,https://sports.<STATE>.betmgm.com/en/sports?op...,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,over,5.5
4,38874-32634-2024-01-26:betmgm:player_assists:a...,BetMGM,Anfernee Simons Under 5.5,-110.0,1.706304e+09,5.5,True,False,Player Assists,player_assists,NaN,NaN,https://sports.<STATE>.betmgm.com/en/sports?op...,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,under,5.5


(695, 18)


### Create Cleaned Sportsbook Data Frame
* Each row will have PrizePicks and show the odds and the line for a couple other highly trusted Sportsbooks.
* This makes it seemless to compare PrizePicks Player Projections to the sentiment of the trusted market.

In [52]:
def filter_for_prizepicks(odds_df):
    # First, sort the DataFrame by the 'market' column, which acts as our unique key.
    sorted_odds_df = odds_df.sort_values(by='market')

    # Group by 'market' to organize our DataFrame into groups based on prediction types.
    grouped = sorted_odds_df.groupby('market')

    # Now, we will keep only the groups that contain at least one 'PrizePicks' entry.
    valid_groups = []
    for name, group in grouped:
        if 'PrizePicks' in group['sports_book_name'].values:
            valid_groups.append(group)
    if len(valid_groups) > 0:
        # Combine all valid groups back into a DataFrame.
        filtered_df = pd.concat(valid_groups)
        filtered_df = filtered_df.groupby(['market', 'sports_book_name']).apply(lambda x: x.reset_index(drop=True)).reset_index(drop=True)
    else:
        filtered_df = pd.DataFrame()
    
    return filtered_df

### Testing the functionality

In [53]:
filtered_odds_df = filter_for_prizepicks(books_df)

In [54]:
filtered_odds_df

,id,sports_book_name,name,price,timestamp,bet_points,is_main,is_live,market_name,market,home_rotation_number,away_rotation_number,deep_link_url,player_id,selection,normalized_selection,selection_line,selection_points
0,62CCECD23CDF,PrizePicks,Anfernee Simons Over 18.5,-137.0,1.706313e+09,18.5,True,False,1st Half Player Fantasy Score (PrizePicks),1st_half_player_fantasy_score_prizepicks_,NaN,NaN,None,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,over,18.5
1,147F7DB13100,PrizePicks,Anfernee Simons Under 18.5,-137.0,1.706313e+09,18.5,True,False,1st Half Player Fantasy Score (PrizePicks),1st_half_player_fantasy_score_prizepicks_,NaN,NaN,None,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,under,18.5
2,DB05A279814F,PrizePicks,Deandre Ayton Over 15.5,-137.0,1.706313e+09,15.5,True,False,1st Half Player Fantasy Score (PrizePicks),1st_half_player_fantasy_score_prizepicks_,NaN,NaN,None,54818F8E80E9,Deandre Ayton,deandre_ayton,over,15.5
3,0960EC441208,PrizePicks,Deandre Ayton Under 15.5,-137.0,1.706313e+09,15.5,True,False,1st Half Player Fantasy Score (PrizePicks),1st_half_player_fantasy_score_prizepicks_,NaN,NaN,None,54818F8E80E9,Deandre Ayton,deandre_ayton,under,15.5
4,EABC944A657F,PrizePicks,Jabari Walker Over 11.5,-137.0,1.706313e+09,11.5,True,False,1st Half Player Fantasy Score (PrizePicks),1st_half_player_fantasy_score_prizepicks_,NaN,NaN,None,677724637AF2,Jabari Walker,jabari_walker,over,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,4F51FEFE6F09,PrizePicks,Deandre Ayton Under 1.5,-137.0,1.706312e+09,1.5,True,False,Player Turnovers,player_turnovers,NaN,NaN,None,54818F8E80E9,Deandre Ayton,deandre_ayton,under,1.5
579,39799625B337,Underdog Fantasy,Deandre Ayton Under 1.5,-137.0,1.706285e+09,1.5,True,False,Player Turnovers,player_turnovers,NaN,NaN,None,54818F8E80E9,Deandre Ayton,deandre_ayton,under,1.5
580,E44A6C61B617,Underdog Fantasy,Deandre Ayton Over 1.5,-137.0,1.706285e+09,1.5,True,False,Player Turnovers,player_turnovers,NaN,NaN,None,54818F8E80E9,Deandre Ayton,deandre_ayton,over,1.5
581,AF7587D74DF2,Underdog Fantasy,Anfernee Simons Under 2.5,-137.0,1.706285e+09,2.5,True,False,Player Turnovers,player_turnovers,NaN,NaN,None,A08DF5DBBEE9,Anfernee Simons,anfernee_simons,under,2.5


### Organize The DF Into Something We Can Scan Easily

In [55]:
def create_cleaned_props_df(filtered_df):
    book_order = ["PrizePicks", "Underdog Fantasy", "Pinnacle", "FanDuel", "Draftkings", "BetMGM"]

    # Create a list to store our new rows
    cleaned_data = []

    # Group by 'market' and then by 'selection' within each group
    for market, market_df in filtered_df.groupby('market'):
        for selection in ['over', 'under']:
            row = {'Player Name': None, 'Selection': selection, 'Market Name': market}
            selection_present = False

            for book in book_order:
                book_df = market_df[(market_df['sports_book_name'] == book) & (market_df['selection_line'] == selection)]
                
                if not book_df.empty:
                    selection_present = True
                    # We assume that there's only one row for each book per selection
                    book_row = book_df.iloc[0]
                    row['Player Name'] = book_row['normalized_selection'].replace('_', ' ').title()
                    row[f'{book} Line Number'] = book_row['bet_points']
                    row[f'{book} Odds'] = book_row['price']
                else:
                    row[f'{book} Line Number'] = '-'
                    row[f'{book} Odds'] = '-'
            
            # If we found at least one entry for this selection, add the row to our list
            if selection_present:
                cleaned_data.append(row)

    # Convert the list of rows into a DataFrame
    cleaned_props_df = pd.DataFrame(cleaned_data)
    return cleaned_props_df

# Then you would call this function, passing the DataFrame you've obtained after applying the filter_for_prizepicks function:
# cleaned_props_df = create_cleaned_props_df(filtered_df)

### For Each Player On A Team, Find All Player Props And Construct A Prop Line And Odds Comparion Table

In [57]:
cleaned_props_df = create_cleaned_props_df(filtered_odds_df)
display(cleaned_props_df)

,Player Name,Selection,Market Name,PrizePicks Line Number,PrizePicks Odds,Underdog Fantasy Line Number,Underdog Fantasy Odds,Pinnacle Line Number,Pinnacle Odds,FanDuel Line Number,FanDuel Odds,Draftkings Line Number,Draftkings Odds,BetMGM Line Number,BetMGM Odds
0,Anfernee Simons,over,1st_half_player_fantasy_score_prizepicks_,18.5,-137.0,-,-,-,-,-,-,-,-,-,-
1,Anfernee Simons,under,1st_half_player_fantasy_score_prizepicks_,18.5,-137.0,-,-,-,-,-,-,-,-,-,-
2,Anfernee Simons,over,1st_half_player_points,12.5,-137.0,-,-,-,-,-,-,-,-,-,-
3,Anfernee Simons,under,1st_half_player_points,12.5,-137.0,-,-,-,-,-,-,-,-,-,-
4,Deandre Ayton,over,1st_half_player_points_+_rebounds_+_assists,13.5,-137.0,-,-,-,-,-,-,-,-,-,-
5,Deandre Ayton,under,1st_half_player_points_+_rebounds_+_assists,13.5,-137.0,-,-,-,-,-,-,-,-,-,-
6,Jabari Walker,over,player_assists,5.5,-137.0,5.5,-137.0,5.5,-101.0,7.5,330.0,-,-,1.5,180.0
7,Jabari Walker,under,player_assists,5.5,-137.0,5.5,-137.0,5.5,-130.0,5.5,-120.0,-,-,1.5,-250.0
8,Deandre Ayton,over,player_defensive_rebounds,7.0,-137.0,-,-,-,-,-,-,-,-,-,-
9,Deandre Ayton,under,player_defensive_rebounds,7.0,-137.0,-,-,-,-,-,-,-,-,-,-


### For Each Player On A Team, Find All Player Props And Construct A Prop Line And Odds Comparion Table